In [1]:
!pip install pymongo[srv]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.1 MB/s eta 0:00:00


In [2]:
!pip install python-dotenv

In [6]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv
load_dotenv('/content/api.env')
mongo_access=os.environ.get("mongodb_access")
# Create a new client and connect to the server
client = MongoClient(mongo_access, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [7]:
db=client["Steam_Game"]
collection=db["Steam_Embedding"]
collection.find()

In [8]:
from sentence_transformers import SentenceTransformer, util
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def get_embedding(text):
    if not isinstance(text, str) or not text.strip():
        print("Skipping invalid text")
        return []
    return embedding_model.encode(text).tolist()


In [16]:
import re
def extract_metadata_tags(text):
    tags = {
        "year": None,
        "price": None,
        "review": None
    }

    year_match = re.search(r"\b(19|20)\d{2}\b", text)
    if year_match:
        tags["year"] = int(year_match.group())

    price_match = re.search(r"\$?(\d+)(\.\d{1,2})?", text)
    if price_match:
        tags["price"] = float(price_match.group(1))

    for sentiment in ["Positive", "Mixed", "Negative"]:
        if sentiment.lower() in text.lower():
            tags["review"] = sentiment
            break

    return tags


In [18]:
def smart_vector_search(user_query, collection, limit=100):

    tags = extract_metadata_tags(user_query)

    # Step 1: Embed query
    query_embedding = get_embedding(user_query)
    if not query_embedding:
        return []

    # Step 2: Run vector search in MongoDB
    pipeline = [
        {
            "$vectorSearch": {
                "index": "default",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 400,
                "limit": limit,
            }
        },
        {"$unset": "embedding"},
        {
            "$project": {
                "_id": 0,
                "name": 1,
                "description": 1,
                "all_reviews": 1,
                "release_date": 1,
                "developer": 1,
                "publisher": 1,
                "price": 1,
                "score": {"$meta": "vectorSearchScore"},
            }
        }
    ]

    results = list(collection.aggregate(pipeline))

    # Step 3: Post-filter by metadata
    filtered_results = []
    for game in results:
        # Check year
        if tags["year"] is not None:
          if not game.get("release_date") or game["release_date"].year != tags["year"]:
            continue


        if tags["price"] is not None:
          if not game.get("price"):
            continue
          try:
            game_price = float(str(game["price"]).replace("$", "").strip())
            if game_price > tags["price"]:
              continue
          except ValueError:
            continue  # Skip games with malformed price


        # Check review
        if tags["review"]:
            if not game.get("all_reviews") or tags["review"].lower() not in game["all_reviews"].lower():
                continue

        # If all filters pass
        filtered_results.append(game)

    return filtered_results[:3] if filtered_results else results[:3]

In [20]:
user_query = "I'm looking for a tactical football game with very positive reviews, released in 2020, developed by an indie studio, and priced under $10"
games = smart_vector_search(user_query, collection)

for game in games:
    print(game)

{'name': 'Easy Ball Game', 'all_reviews': 'Positive', 'release_date': datetime.datetime(2020, 9, 20, 0, 0), 'developer': 'Eros Rossi', 'publisher': 'Eros Rossi', 'price': '$6.99 ', 'description': 'defy gravity game ball', 'score': 0.7655874490737915}
{'name': 'Time Over', 'all_reviews': 'Very Positive', 'release_date': datetime.datetime(2020, 10, 26, 0, 0), 'developer': 'Roitman', 'publisher': 'Cow Games', 'price': '$2.99 ', 'description': 'unlock secret unparalleled success power teamwork', 'score': 0.7648532390594482}
{'name': 'DEFENDit', 'all_reviews': 'Mostly Positive', 'release_date': datetime.datetime(2020, 4, 29, 0, 0), 'developer': 'edybtt Studios', 'publisher': 'edybtt Studios', 'price': '$0', 'description': 'defend sure defend base last enemy try luck free play', 'score': 0.7647790908813477}
{'name': "Backyard Soccer '98", 'all_reviews': 'Very Positive', 'release_date': datetime.datetime(2020, 10, 6, 0, 0), 'developer': 'Mega Cat Studios', 'publisher': 'Playground Productions

In [22]:
user_query="Counter Strike"
games = smart_vector_search(user_query, collection)

for game in games:
    print(game)

{'name': 'Counter-Strike 2', 'all_reviews': 'Very Positive', 'release_date': datetime.datetime(2021, 3, 17, 0, 0), 'developer': 'Valve', 'publisher': 'Valve', 'price': '$0', 'description': 'two decades counterstrike offered elite competitive experience one shaped millions players across globe next chapter cs story begin counterstrike 2', 'score': 0.8244240283966064}
{'name': 'Strike.is: The Game', 'all_reviews': 'Mostly Positive', 'release_date': datetime.datetime(2022, 5, 25, 0, 0), 'developer': 'Freakinware Limited', 'publisher': 'Freakinware Limited', 'price': '$0', 'description': 'load gun dominate best multiplayer mobile shooter game', 'score': 0.7655930519104004}
{'name': 'Counter Spell', 'all_reviews': 'Very Positive', 'release_date': datetime.datetime(2021, 11, 11, 0, 0), 'developer': 'Abnormal Software', 'publisher': 'Abnormal Software', 'price': '$0.14', 'description': 'counter spell combines elements classic role playing games modern stealthbased shooters single player multi

In [23]:
from google.genai import types

def generate_response(user_query, retrieved_games, client):
    """
    Generate a recommendation response using Gemini Flash API.

    Args:
        user_query (str): The original user query.
        retrieved_games (list): List of game metadata dicts.
        client: The initialized Google Generative AI client.

    Returns:
        str: The generated recommendation text.
    """
    context = "\n".join([
        f"{g['name']}: {g['description']}: {g['all_reviews']}: {g['release_date']}:{g['developer']} :{g['publisher']}: {g['price']}"
        for g in retrieved_games
    ])

    prompt = f"""
You are a game recommendation agent. Your task is to provide engaging and convincing recommendations to users based on their queries and the following retrieved game information.

User Query: {user_query}

Relevant Games:
{context}

Provide a detailed recommendation that:
- Highlights the most appealing aspects of the games.
- Connects the games to the user's query.
- Uses persuasive language and your own knowledge to captivate the user.
- Includes relevant information such as gameplay, reviews, release date, publisher and price.
- Make the user want to play the game.
"""

    # Use client.models.generate_content() instead of client.generate_content()
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )

    return response.text

In [24]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.4 MB/s eta 0:00:00


In [25]:
from google import genai

gemini_api_key = os.environ.get("gemini_api_key")
client = genai.Client(api_key=gemini_api_key)

In [ ]:
from google.genai import types
function_declarations = [
    {
        "name": "vector_search",
        "description": "Search games by user query",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "User input that includes a description of games they want"
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "chit_chat",
        "description": "Chit chat message of users",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "User input message that is common chit-chat"
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "end_chat",
        "description": "End chat session",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "End the chat session and further information if needed"
                }
            },
            "required": ["query"]
        }
    }

]

tools = types.Tool(function_declarations=function_declarations)
config = types.GenerateContentConfig(tools=[tools])

# Send query to Gemini Flash

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=user_query,
    config=config,
)

# Process Gemini Flash function call and generate final response
def process_response(response, user_query, collection, client):
    candidate = response.candidates[0]

    if candidate.content.parts and candidate.content.parts[0].function_call:
        function_call = candidate.content.parts[0].function_call
        print(f"🔧 Function to call: {function_call.name}")
        print(f"📥 Arguments: {function_call.args}")

        if function_call.name == "vector_search":
            results = smart_vector_search(function_call.args["query"], collection)
        elif function_call.name == "chit-chat":
            chit_chat_response=client.models.generate_content(
                model="gemini-2.0-flash",
                contents=function_call.args["query"],
            )
            results=chit_chat_response.text
        else:
            end_response=client.models.generate_content(
                model="gemini-2.0-flash",
                contents=function_call.args["query"],
            )
            results=end_response.text

        # Use local model to generate the response
        return generate_response(user_query, results, client)

# Final response
user_query = "Recommend shooting games in 2023 with positive reviews"
final_response = process_response(response, user_query, collection,client)
print(final_response)

🔧 Function to call: vector_search_description
📥 Arguments: {'query': 'shooting games in 2023 with positive reviews'}
Looking for a thrilling shooting game from 2023 with glowing reviews? Look no further! I've got a fantastic lineup tailored just for you.

First up, prepare for an all-out alien invasion in **EARTH DEFENSE FORCE 5**. This isn't just any shooter; it's an arcade-style, humanity-saving extravaganza! Released in June 2023 by D3PUBLISHER, you'll step into the boots of an EDF soldier and battle endless hordes of immense, otherworldly enemies to restore peace to Earth. Players on Steam have given it Very Positive reviews, praising its non-stop action and addictive gameplay. And the best part? You can grab it now for just $3.32!

Alternatively, if you like cute shooter games, you should check out **Lovely Planet 2: April Skies**. As the sequel to the critically acclaimed 2014 game, this title will put you on a gun ballet adventure. This game was released in July 2023 by tinyBuil